In [1]:
from utils import DataLoader

eeg = DataLoader(1,(1,9))

done with p


In [8]:
eegtuple = eeg.load_data_for_nn(participant=1, sigtype='eeg')

In [9]:
eegtuple[0].shape, eegtuple[1].shape, eegtuple[2].shape

((4115, 32, 294), (294, 3), (294, 3))

In [11]:
import numpy as np

In [12]:
first = True
for i, array in enumerate(np.dsplit(eegtuple[0], 294)):   
    if eegtuple[2][i,2] != 0:
        if first:
            filtered = array
            weights = eegtuple[1][i]
            textures = eegtuple[2][i]
            first = False
        else:
            filtered = np.dstack((filtered, array))
            weights = np.vstack((weights, eegtuple[1][i]))
            textures = np.vstack((textures, eegtuple[2][i]))

In [13]:
filtered.shape, weights.shape, textures.shape

((4115, 32, 220), (220, 3), (220, 3))

In [14]:
shuffler = np.random.permutation(range(220))
filtered = filtered[:,:,shuffler]
weights = weights[shuffler]
textures = textures[shuffler]

In [22]:
from AdversarialCNN import AdversarialCNN

net = AdversarialCNN(chans=32, samples=4115, n_output=3, n_nuisance=3, architecture='DeepConvNet', adversarial=False, lam=0.01)

In [16]:
eeg_training = np.reshape(filtered[:,:,:176], (4115, 32, 176, 1))
eeg_testing = np.reshape(filtered[:,:,176:], (4115, 32, 44, 1))
weight_training = weights[:176,:]
weight_testing = weights[176:,:]
texture_training = textures[:176,:]
texture_testing = textures[176:,:]

In [17]:
eeg_training = np.moveaxis(eeg_training, 0, 2)
eeg_training = np.moveaxis(eeg_training, 0, 1)
eeg_testing = np.moveaxis(eeg_testing, 0, 2)
eeg_testing = np.moveaxis(eeg_testing, 0, 1)
eeg_training.shape, eeg_testing.shape

((176, 32, 4115, 1), (44, 32, 4115, 1))

In [18]:

train_set = (eeg_training, weight_training, texture_training)
test_set = (eeg_testing, weight_testing, texture_testing)

In [23]:
net.train(train_set, test_set, log = 'logdir', epochs=500, batch_size=1)

Epoch 1/500
/home/rohass/anaconda3/envs/deep/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/rohass/anaconda3/envs/deep/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Train - [Loss: 7.976208] - [CLA loss: 7.976208, acc: 37.50%] - [ADV loss: 0.000001, acc: 100.00%]
Validation - [Loss: 6.458746] - [CLA loss: 6.458746, acc: 38.64%] - [ADV loss: 0.000000, acc: 100.00%]
Epoch 2/500
Train - [Loss: 7.566335] - [CLA loss: 7.566335, acc: 53.98%] - [ADV loss: 0.000000, acc: 100.00%]
Validation - [Loss: 10.074579] - [CLA loss: 10.074579, 

KeyboardInterrupt: 